# 1导入库与数据

In [3]:
import pandas as pd
import os
import numpy as np

In [5]:
file_path = r'C:\Users\a\.spyder-py3\tf-notebooks\residential_electricity'
#Windows系统中，路径中的反斜杠\是转义字符，所以你需要使用双反斜杠\\或者前置r来创建一个原始字符串。
os.chdir(file_path)
df = pd.read_csv("data.csv",parse_dates = [1],index_col= [0,1])
#index_col设置双重索引
df.head()

KWH
CONS_NO DATA_DATE       
1       2015-01-01  6.68
        2015-01-02  2.50
        2015-01-03  5.20
        2015-01-04  4.17
        2015-01-05  4.89

In [6]:
replace_dict = {"NA":np.nan,np.nan:np.nan}
df['KWH'].fillna(replace_dict,inplace = True)

In [7]:
df.isnull().sum()

KWH    1018
dtype: int64

# 2数据预处理

In [8]:
df = df.unstack().KWH
#对内层索引转换为列，通常用于时间序列数据将KWH这列作为结果

In [9]:
type(df.iloc[1,1])

numpy.float64

In [10]:
df.head()

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,6.68,2.50,5.20,4.17,4.89,5.26,4.11,3.70,3.74,5.83,...,2.37,3.12,2.02,1.74,2.53,3.18,3.29,2.77,3.81,3.62
2,1.22,0.65,1.14,1.04,1.33,1.02,0.08,0.09,0.09,0.09,...,0.97,0.95,0.93,0.92,0.92,1.99,0.96,1.36,1.33,1.40
3,7.35,6.65,7.76,4.02,4.68,7.06,6.51,7.00,6.53,4.94,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.62,1.92,0.65,1.30,0.71,1.36,0.89,1.24,0.75,1.67,...,1.14,1.06,0.96,1.12,0.86,0.76,0.75,1.49,0.86,1.08
5,2.58,2.60,2.36,1.83,2.05,1.97,1.98,1.44,2.02,2.33,...,2.11,2.58,2.22,1.96,1.71,2.23,2.69,2.27,2.52,2.10


## 异常值

In [11]:
for col in range(df.shape[1]):
    col_mean = df.iloc[:,col].mean()
    col_std =df.iloc[:,col].std()
    mask = (df.iloc[:,col]-col_mean).abs()>3*col_std
    df.iloc[:,col][mask] = np.nan
    #根据条件矩阵进行赋值时，只有布尔矩阵中对应位置为 True 的元素会被替换。

In [12]:
df.head()

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,6.68,2.50,5.20,4.17,4.89,5.26,4.11,3.70,3.74,5.83,...,2.37,3.12,2.02,1.74,2.53,3.18,3.29,2.77,3.81,3.62
2,1.22,0.65,1.14,1.04,1.33,1.02,0.08,0.09,0.09,0.09,...,0.97,0.95,0.93,0.92,0.92,1.99,0.96,1.36,1.33,1.40
3,7.35,6.65,7.76,4.02,4.68,7.06,6.51,7.00,6.53,4.94,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0.62,1.92,0.65,1.30,0.71,1.36,0.89,1.24,0.75,1.67,...,1.14,1.06,0.96,1.12,0.86,0.76,0.75,1.49,0.86,1.08
5,2.58,2.60,2.36,1.83,2.05,1.97,1.98,1.44,2.02,2.33,...,2.11,2.58,2.22,1.96,1.71,2.23,2.69,2.27,2.52,2.10


In [13]:
df.max(axis =1)

CONS_NO
1      25.16
2      15.46
3      27.00
4       4.70
5      16.50
       ...  
196     6.28
197    50.40
198    24.89
199    11.83
200     0.00
Length: 200, dtype: float64

In [14]:
df.agg(['max','min','median','sum','var','skew','kurt'],axis=1)

,max,min,median,sum,var,skew,kurt
CONS_NO,,,,,,,
1,25.16,0.0,3.120,3065.93,11.762377,3.040316,11.211566
2,15.46,0.0,1.790,1695.10,3.926000,3.420053,14.341789
3,27.00,0.0,3.600,2803.25,14.945491,1.582819,5.123261
4,4.70,0.0,1.000,860.84,0.301526,1.763758,4.600405
5,16.50,0.0,2.270,2374.45,6.194531,2.502617,6.188802
...,...,...,...,...,...,...,...
196,6.28,0.0,1.170,894.53,0.974862,2.133523,7.439265
197,50.40,0.0,0.765,2194.90,51.468509,3.810612,15.842375
198,24.89,0.0,3.665,2770.84,9.502098,3.238847,14.851859


## 差分处理

In [15]:
df.diff(axis = 1).agg(['max','min','median','sum','var','skew','kurt'])

DATA_DATE,2015-01-01,2015-01-02,2015-01-03,2015-01-04,2015-01-05,2015-01-06,2015-01-07,2015-01-08,2015-01-09,2015-01-10,...,2017-01-29,2017-01-30,2017-01-31,2017-02-01,2017-02-02,2017-02-03,2017-02-04,2017-02-05,2017-02-06,2017-02-07
max,NaN,5.360000,5.710000,5.320000,3.850000,8.880000,5.200000,4.870000,3.440000,4.910000,...,10.510000,4.360000,5.060000,11.640000,6.590000,6.350000,7.580000,5.100000,4.920000,11.350000
min,NaN,-4.200000,-5.200000,-5.460000,-4.910000,-3.380000,-6.180000,-4.240000,-6.420000,-8.480000,...,-6.460000,-5.020000,-5.430000,-3.050000,-9.770000,-9.450000,-5.900000,-5.380000,-9.090000,-6.900000
median,NaN,0.000000,0.000000,0.000000,0.000000,0.005000,-0.030000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
sum,0.0,15.080000,25.160000,-26.600000,-17.810000,54.800000,-47.320000,-14.560000,-2.550000,11.540000,...,-33.000000,-19.170000,-8.730000,68.630000,-4.800000,28.860000,3.660000,-10.950000,-7.260000,19.110000
var,NaN,1.529140,1.453966,1.842255,1.289004,1.510381,1.355740,0.969286,1.374143,1.537225,...,1.944687,1.133146,1.205654,2.863745,2.644233,2.447967,1.896841,1.449804,1.864745,2.147470
skew,NaN,0.664766,0.313423,-0.073182,-0.137913,2.217920,-0.272209,-0.125547,-1.302175,-1.697730,...,1.474367,-0.546191,-0.335455,3.442935,-2.190073,-0.754902,0.657414,-0.006649,-1.652249,2.127068
kurt,NaN,4.612046,4.160558,3.234514,3.750176,13.244584,6.564531,5.967313,8.744256,12.871007,...,20.783532,6.768244,8.153275,17.756790,13.953656,9.051757,6.685429,4.797595,11.431051,21.127475


## 分位数计算

In [16]:
df_quantile = df.quantile([.05],axis = 1)

In [17]:
df_quantile

CONS_NO,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
0.05,1.63,0.6905,0.0,0.58,1.5005,2.12,0.7625,1.107,0.0,0.0,...,0.0605,1.614,0.04,0.0,0.0205,0.0,0.0,1.13,0.15,0.0


## 按周求和

In [18]:
df.columns.isocalendar().week

DATA_DATE
2015-01-01    1
2015-01-02    1
2015-01-03    1
2015-01-04    1
2015-01-05    2
             ..
2017-02-03    5
2017-02-04    5
2017-02-05    5
2017-02-06    6
2017-02-07    6
Name: week, Length: 746, dtype: UInt32

In [19]:
print(df.dtypes)

DATA_DATE
2015-01-01    float64
2015-01-02    float64
2015-01-03    float64
2015-01-04    float64
2015-01-05    float64
               ...   
2017-02-03    float64
2017-02-04    float64
2017-02-05    float64
2017-02-06    float64
2017-02-07    float64
Length: 746, dtype: object


In [20]:

df_week_diff =df.groupby(df.columns.isocalendar().week,axis=1).sum().diff(axis =1)


In [21]:
df_week_diff

week,1,2,3,4,5,6,7,8,9,10,...,44,45,46,47,48,49,50,51,52,53
CONS_NO,,,,,,,,,,,,,,,,,,,,,
1,NaN,13.32,-9.07,6.56,3.82,-23.02,-14.09,-7.93,13.87,1.44,...,-5.12,10.93,2.46,2.06,2.11,-3.89,-0.52,3.99,-4.67,-14.57
2,NaN,-0.36,2.79,1.40,2.16,-20.77,3.14,-5.35,-2.57,7.79,...,-3.41,6.38,1.24,-1.40,1.49,0.01,-1.59,-1.79,0.22,-13.07
3,NaN,13.99,9.44,-4.62,6.38,-29.93,29.13,-16.67,3.21,9.24,...,-2.74,40.38,-5.05,-18.38,0.91,2.75,3.76,-13.56,1.55,2.45
4,NaN,2.53,1.29,-0.23,-1.79,-4.59,-3.75,7.18,-1.76,-2.52,...,-2.76,4.19,1.34,-1.85,3.70,-3.27,-1.23,2.85,-4.58,-3.23
5,NaN,3.87,0.23,1.94,1.86,-20.07,0.63,4.84,-7.60,1.69,...,-2.80,11.52,0.37,-1.72,-2.11,0.69,1.28,2.61,-4.77,-13.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,NaN,0.58,3.32,-20.53,-1.69,-8.31,-3.20,5.63,5.81,0.56,...,-3.61,12.75,-5.16,-1.19,6.42,-6.45,-0.51,5.29,0.18,-11.12
197,NaN,25.92,-9.47,3.33,-28.55,-10.48,3.13,-1.02,-2.29,0.45,...,-8.11,17.01,-14.12,4.29,34.12,-0.94,-14.37,25.47,-4.43,-51.54
198,NaN,10.00,-13.66,-2.75,-20.19,-20.37,-10.69,-5.39,17.29,6.99,...,-0.14,2.77,-12.56,28.92,-4.63,5.16,-0.47,3.94,-2.63,-25.87
